In [ ]:
from datasets import load_dataset

dataset = load_dataset("amazon_reviews_multi", "es")

In [ ]:
dataset["validation"][0]

# Client setup

In [ ]:
import rubrix
import os

In [ ]:
rubrix.init(api_url=os.environ["RUBRIX_API_URL"], api_key=os.environ["RUBRIX_API_KEY"])

# Store labeled dataset for initial exploration


Here we will store the validation dataset with labels, inputs and associated metadata.


In [ ]:
records = []

for record in dataset['validation']:
    records.append(
        rubrix.TextClassificationRecord(
            inputs={
                "review_body": record["review_body"], 
                "review_title": record["review_title"]
            },
            annotation=record["stars"],
            annotation_agent="test",
            metadata={
                "product_category": record["product_category"],
                "reviewer_id": record["reviewer_id"]
            },
            id=record["product_id"],
        )
    )

In [ ]:
records[0:5]

In [ ]:
rubrix.log(
    name="amazon_sentiment_es_validation_ds_multifield", 
    recTextClassificationRecordecords, 
    tags={ 
        "type":"sentiment classifier",
        "lang": "spanish",
        "description": "Spanish sentiment classifier with `multifield inputs` (title and body)"
    }, 
    chunk_size=100
)

# Store predictions and labels with a pretrained model for error analysis

Here we will use a pretrained transformer from huggingface Hub for analysing its quality with an unseen dataset

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

CHKPT = 'mrm8488/electricidad-small-finetuned-muchocine'
model = AutoModelForSequenceClassification.from_pretrained(CHKPT)
tokenizer = AutoTokenizer.from_pretrained(CHKPT)

from transformers import pipeline
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, return_all_scores=True)

In [ ]:
def get_stars(label):
    return len([e for e in label if e != ' '])

In [ ]:
[{"class":get_stars(e['label']), "confidence": e['score']} for e in classifier("Una buena película, sin más.")[0]]

In [ ]:
def create_record_with_preds(idx, inputs, label, metadata, prediction):
    return TextClassificationRecord.from_dict({
         "idx": idx,
         "inputs": inputs,
         "annotation": {
             "agent": "test",
             "labels": [{"class": label}]
         },
         "prediction": prediction,
         "metadata": metadata
    })

In [ ]:
records = []

for record in dataset['validation']:
    records.append(
        rubrix.TextClassificationRecord(
            inputs={
                "review_body": record['review_body'],
                "review_title": record['review_title'],
            },
            prediction=[(get_stars(e['label']), e['score']) for e in classifier(record['review_body'])[0]],
            prediction_agent=CHKPT,
            annotation=record["stars"],
            annotation_agent="test",
            metadata={
                "product_category": record["product_category"],
                "reviewer_id": record["reviewer_id"]
            },
            id=record["product_id"],
        )
    )

In [ ]:
len(records)

In [ ]:
rubrix.log(
    name="amazon_sentiment_es_validation_ds_multifield", 
    records=records, 
    tags={
        "env": "test", 
        "model": CHKPT, 
        "type": "evaluation dataset"
    }, 
    chunk_size=200
)

# Store only predictions for model monitoring

Here we will use a pretrained transformer from huggingface Hub for monitoring the model predictions over time.

In [ ]:
def create_record_only_preds(idx, inputs, metadata, prediction):
    return TextClassificationRecord.from_dict({
         "idx": idx,
         "inputs": inputs,

         "prediction": prediction,
         "metadata": metadata
    })

In [ ]:
records = []

for record in dataset['validation']:
    records.append(
        rubrix.TextClassificationRecord(
            inputs={
                "review_body": record['review_body'],
                "review_title": record['review_title']
            },
            prediction=[(get_stars(e['label']), e['score']) for e in classifier(record['review_body'])[0]],
            prediction_agent=CHKPT,
            metadata={
                "product_category": record["product_category"],
                "reviewer_id": record["reviewer_id"]
            },
            id=record["product_id"],
        )
    )

In [ ]:
rubrix.log(
    name="amazon_sentiment_es_validation_ds_multifield", 
    records=records, 
    tags={
        "env": "test", 
        "model": CHKPT, 
        "type": "evaluation dataset"
    }, 
    chunk_size=200
)